In [22]:
import numpy as np
import pandas as pd
import re
import os

#Human rRNA contigs and their lengths, as generated by Alex's fastawork.py
contig_list = {'18s': 1869, '28s': 5070, '5.8s': 157, '5s': 121}

def read_pileup_data(filepath, protein_list=None):
    '''
    Accepts filepath to .csv file containing pileup data (RPMS - reads per million) of a given contig, 
    and optionally, a list of protein names to search for in the dataset. 
    Outputs .csv file of pileup RPMs binned by nucleotide position in the contig
    '''
    global contig_list
    
    #Looks for contig listed in the filename
    filename = filepath.split('/')[-1]
    contig = filename.split('_')[0]
    print(f"Detected {contig} as contig used in filename")
    
    #Read .csv file of pileup data
    pileup_data = pd.read_csv(filepath,index_col=0)

    #Cleans up column names by removing "_sorted_CONTIG_[contig]" trailer
    pileup_data.columns = [re.search("(ENCSR[A-Za-z0-9]*_[A-Za-z0-9]*_rep[0-9]*)_sorted",column).group(1) for column in pileup_data.columns]
    
    if protein_list != None:
        #Filter dataset columns to only include those containing the proteins listed in "protein_list"
        protein_regex = '_|'.join(protein_list)
        filtered_pileup = pileup_data.filter(regex=protein_regex,axis=1)
    else:
        filtered_pileup=pileup_data

    #Infers contig length by number of rows in the input pileup data. 
    #Checks it against the length listed in the global "contig_list" variable
    contig_length = len(pileup_data.index)
    assert contig_length == contig_list[contig], "File row length does not match contig length"
    
    #Generates 20nt bins and averages them to generate an output Dataframe of binned RPMs
    groups = np.arange(1,contig_length+1,1/20)[0:contig_length]
    groups = np.floor(groups)
    pileup_group = filtered_pileup.groupby(groups)
    avg_pileup = pileup_group.mean().round(0)
    avg_pileup.index.name = "20nt_group"
    avg_pileup.columns.name = f"sample ({contig})"

    return avg_pileup
    # #group by 20nt and average. Last group only contains 10nt
    # groups = np.arange(1,95,1/20)[0:1870]
    # groups = np.floor(groups)
    # pileup_group = pileup_data.groupby(groups)
    # avg_pileup = pileup_group.mean().round(0)
    # avg_pileup.index.name = "20nt_group"
    # avg_pileup.columns.name = "sample"
    # avg_pileup.to_csv("./18S_average_pileup.csv")

    # with open("./18S_average_pileup.pkl", "wb") as file:
    #     pickle.dump(avg_pileup, file)


if __name__ == "__main__":
    path = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/"
    file_list = os.listdir(path)
    file_list = list(filter(lambda filename: filename.endswith("_pileup_table.csv"), file_list))
    path_file_list = [path+filename for filename in file_list]
    
    average_pileup_output_path = path +"average_pileup_output/"
    if not os.path.exists(average_pileup_output_path):
        os.mkdir(average_pileup_output_path)
    
    for filepath in path_file_list:
        avg_pileup = read_pileup_data(filepath)
        output_filepath = filepath.split('/')[-1].replace('_pileup_table.csv','_average_pileup.csv')
        avg_pileup.to_csv(average_pileup_output_path + output_filepath)



/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/18s_pileup_table.csv
Detected 18s as contig used in filename
/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/28s_pileup_table.csv
Detected 28s as contig used in filename
/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/5s_pileup_table.csv
Detected 5s as contig used in filename
/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/5.8s_pileup_table.csv
Detected 5.8s as contig used in filename


In [ ]:
port sliding window code
clear pileup_data column names
drop first "unamed 0" column
Figure out which proteins are controls using Anthony's list of accession numbers

LONG-TERM:
-Learn how to port Anthony's R code to Python

Overview: eCLIP (for yourself - don't present)
Pipeline - Anthony's slide
Talk about filtering + pileup step (what did we filter, show equation on how reads per million were calculated)
Stats (if we get there)
Future directions